In [19]:
from __future__ import absolute_import
import numpy as nm
from sfepy.discrete.fem import Mesh
from sfepy.discrete.fem.meshio import UserMeshIO
import csv
import numpy as np

In [20]:
def mesh_hook(mesh, mode):
    """
    Generate the 1D mesh.
    """
    if mode == 'read':
        n_nod = 101

        coors = nm.linspace(0.0, 1.0, n_nod).reshape((n_nod, 1))
        conn = nm.arange(n_nod, dtype=nm.int32).repeat(2)[1:-1].reshape((-1, 2))
        mat_ids = nm.zeros(n_nod - 1, dtype=nm.int32)
        descs = ['1_2']

        mesh = Mesh.from_data('laplace_1d', coors, None,
                              [conn], [mat_ids], descs)
        return mesh

    elif mode == 'write':
        pass

In [21]:
def get_coef(ts, coors, mode=None, **kwargs):
    if mode == 'qp':
        x = coors[:, 0]

        val = 0.1 + nm.sin(2 * nm.pi * x)**2
        val.shape = (coors.shape[0], 1, 1)

        return {'val' : val}

In [22]:
filename_mesh = UserMeshIO(mesh_hook)

filename_mesh

UserMeshIO

In [23]:
n_nod = 101

coors = nm.linspace(0.0, 1.0, n_nod).reshape((n_nod, 1))
conn = nm.arange(n_nod, dtype=nm.int32).repeat(2)[1:-1].reshape((-1, 2))
mat_ids = nm.zeros(n_nod - 1, dtype=nm.int32)
descs = ['1_2']

mesh = Mesh.from_data('laplace_1d', coors, None,
                        [conn], [mat_ids], descs)

mesh

Mesh:laplace_1d

In [24]:
# Read boundary condition data from CSV
def read_temp_and_hcoeff_from_csv(filename="t_o_and_h_o.csv"):
    """Read temperature and heat transfer coefficient data from CSV file."""
    t_o, h_o = [], []
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            if len(row) >= 2:
                t_o.append(float(row[0].strip()))
                h_o.append(float(row[1].strip()))
    return t_o, h_o

t_o, h_o = read_temp_and_hcoeff_from_csv()


In [25]:
h_o[len(h_o)-1]

17.2

In [26]:
def get_coef(ts, coors, mode=None, **kwargs):
    if mode == 'qp':
        x = coors[:, 0]

        val = 0.1 + nm.sin(2 * nm.pi * x)**2
        val.shape = (coors.shape[0], 1, 1)

        return {'val' : val}

In [27]:
coors = np.array([[0.0], [0.5], [1.0]])  # Example coordinates
result = get_coef(None, coors, mode='qp')
print(result)


{'val': array([[[0.1]],

       [[0.1]],

       [[0.1]]])}


In [28]:
def read_temp_and_hcoeff_from_csv(filename="t_o_and_h_o.csv"):
    """Read temperature and heat transfer coefficient data from CSV file."""
    t_o, h_o = [], []
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            if len(row) >= 2:
                t_o.append(float(row[0].strip()))
                h_o.append(float(row[1].strip()))
    return t_o, h_o

t_o, h_o = read_temp_and_hcoeff_from_csv()
nr_hour = len(t_o)  # Number of hours

In [29]:
def get_bc_values(ts, coor, mode=None, region_name=None, **kwargs):
    """
    Time-dependent boundary condition values.
    """
    if mode != 'qp': return {}
    # Boundary conditions
    t_i = 0.0         # Inner temperature [°C]
    h_i = 99.75       # Inner heat transfer coefficient [W/m2K]
    
    hour = ts.time / 3600  # Current hour
    hour_idx = min(int(hour), nr_hour - 1)  # Index for current hour
    
    nqp = coor.shape[0]
    val = nm.zeros((nqp, 1, 2), dtype=nm.float64)  # h and T_inf
    
    if region_name == 'Gamma_Left':
        # Left boundary (outer surface)
        val[:, 0, 0] = h_o[hour_idx]  # h value
        val[:, 0, 1] = t_o[hour_idx]   # T_inf value
    elif region_name == 'Gamma_Right':
        # Right boundary (inner surface)
        val[:, 0, 0] = h_i  # h value
        val[:, 0, 1] = t_i   # T_inf value
    
    return {'val': val}

In [30]:
class TimeStruct:
    def __init__(self, time):
        self.time = time

ts = TimeStruct(7200)  # 2 hours into the simulation
coor = np.array([[0.0], [0.5], [1.0]])  # Example coordinates
region_name = 'Gamma_Left'  # Specify the boundary

result = get_bc_values(ts, coor, mode='qp', region_name=region_name)
print(result)

{'val': array([[[31.90734191, -4.1       ]],

       [[31.90734191, -4.1       ]],

       [[31.90734191, -4.1       ]]])}


In [31]:
nr_hour = len(t_o)  # Number of hours
print(nr_hour)
# Time parameters
dt = 10.0  # Time step in seconds
nh = int(3600 / dt)  # Number of time steps per hour
print(nh)
total_time = nr_hour * 3600  # Total simulation time in seconds
print(total_time)
n_step = nr_hour * nh  # Total number of time steps
print(n_step)

3672
360
13219200
1321920


In [32]:
def get_h_o(ts, coors, mode=None, **kwargs):
    """
    Time-dependent heat transfer coefficient values (h_o).
    """
    if mode != 'qp' or coors is None:
        return {}

    hour_idx = min(int(ts.time / 3600), len(h_o) - 1)  # Ensure valid index
    val = nm.full((coors.shape[0], 1, 1), h_o[hour_idx], dtype=nm.float64)  # Correct shape

    return {'val': val}

def get_t_o(ts, coors, mode=None, **kwargs):
    """
    Time-dependent ambient temperature values (t_o).
    """
    if mode != 'qp' or coors is None:
        return {}

    hour_idx = min(int(ts.time / 3600), len(t_o) - 1)  # Ensure valid index
    val = nm.full((coors.shape[0], 1, 1), t_o[hour_idx], dtype=nm.float64)  # Correct shape

    return {'val': val}

In [ ]:
def test_get_h_o_with_logging():
    class MockTimeStep:
        def __init__(self, time):
            self.time = time

    h_o_test = [10.0, 15.0, 20.0, 25.0]  # Example hourly values
    coors = nm.array([[0.0], [0.1]])  # Example coordinates

    def get_h_o_override(ts, coors, mode=None, h_o=h_o_test, **kwargs):
        if mode != 'qp' or coors is None:
            return {}

        hour_idx = min(int(ts.time / 3600), len(h_o) - 1)
        val = nm.full((coors.shape[0], 1, 1), h_o[hour_idx], dtype=nm.float64)

        return {'val': val}

    print("Testing get_h_o() at different timestamps:\n")
    
    timestamps = [
        0,  # Start of simulation
        3600,  # Full 1 hour
        7200,  # Full 2 hours
        10800,  # Full 3 hours
        5400,  # Between 1h and 2h (should still use 1-hour value)
        9000,  # Between 2h and 3h (should still use 2-hour value)
    ]

    for time in timestamps:
        ts = MockTimeStep(time=time)
        result = get_h_o_override(ts, coors, mode='qp')
        
        expected_hour_idx = min(int(time / 3600), len(h_o_test) - 1)
        expected_value = h_o_test[expected_hour_idx]

        print(f"Time: {time} sec (Hour {time // 3600}), Expected: {expected_value}, Received: {result['val'][0,0,0]}")

        assert nm.all(result['val'] == expected_value), f"Failed at time {time}"

    print("\ntest_get_h_o_with_logging passed successfully!")

test_get_h_o_with_logging()


Testing get_h_o() at different timestamps:

Time: 0 sec (Hour 0), Expected: 10.0, Received: 10.0
Time: 3600 sec (Hour 1), Expected: 15.0, Received: 15.0
Time: 7200 sec (Hour 2), Expected: 20.0, Received: 20.0
Time: 10800 sec (Hour 3), Expected: 25.0, Received: 25.0
Time: 5400 sec (Hour 1), Expected: 15.0, Received: 15.0
Time: 9000 sec (Hour 2), Expected: 20.0, Received: 20.0

test_get_h_o_with_logging passed successfully!


In [39]:
# Define test parameters
d_ins = 0.1       # Insulation thickness [m]
dx = 0.005        # Cell size [m]
n_el = int(d_ins / dx)  # Number of elements
nodes = n_el + 1  # Number of nodes

print("nodes =", nodes)


nodes = 21


In [40]:
def mesh_hook(mesh, mode):
    """Generate the 1D mesh."""
    if mode == 'read':
        coors = nm.linspace(0.0, d_ins, nodes).reshape((nodes, 1))
        conn = nm.arange(nodes, dtype=nm.int32).repeat(2)[1:-1].reshape((-1, 2))
        mat_ids = nm.zeros(nodes - 1, dtype=nm.int32)
        descs = ['1_2']

        mesh = Mesh.from_data('heat_1d', coors, None,
                             [conn], [mat_ids], descs)
        return mesh
    elif mode == 'write':
        pass

In [42]:


# Function wrapper for testing
def test_mesh_hook():
    mesh = mesh_hook(None, 'read')  # Generate mesh
    coors = mesh.coors  # Get coordinates
    conn = mesh.get_conn('1_2')  # Get connectivity

    # Correct way to access material IDs:
    mat_ids = mesh.cmesh.cell_groups  # Returns array of material group assignments

    print("Mesh generated successfully!")
    print(f"Number of nodes: {coors.shape[0]}")
    print(f"Coordinates:\n{coors}")
    print(f"Connectivity:\n{conn}")
    print(f"Material IDs:\n{mat_ids}")

# Run the test
test_mesh_hook()


Mesh generated successfully!
Number of nodes: 21
Coordinates:
[[0.   ]
 [0.005]
 [0.01 ]
 [0.015]
 [0.02 ]
 [0.025]
 [0.03 ]
 [0.035]
 [0.04 ]
 [0.045]
 [0.05 ]
 [0.055]
 [0.06 ]
 [0.065]
 [0.07 ]
 [0.075]
 [0.08 ]
 [0.085]
 [0.09 ]
 [0.095]
 [0.1  ]]
Connectivity:
[[ 0  1]
 [ 1  2]
 [ 2  3]
 [ 3  4]
 [ 4  5]
 [ 5  6]
 [ 6  7]
 [ 7  8]
 [ 8  9]
 [ 9 10]
 [10 11]
 [11 12]
 [12 13]
 [13 14]
 [14 15]
 [15 16]
 [16 17]
 [17 18]
 [18 19]
 [19 20]]
Material IDs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
